# Variational Autoencoders in edward2

There is a script written with tfp: 

https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/vae.py

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [2]:
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
Variable = tf.Variable
import tensorflow_probability as tfp
tfd = tfp.distributions
ed = tfp.edward2

print('TensorFlow version {}, TF Probability version {}.'.format(tf.__version__, tfp.__version__))

TensorFlow version 1.10.0, TF Probability version 0.4.0.


In [77]:
def generative_model():
    d=10
    z = ed.Normal(loc=tf.zeros([d]), scale=tf.ones([d]), name='z')
    gen_net = tf.keras.Sequential([
        tf.keras.layers.Dense(100, activation=tf.nn.relu),
        tf.keras.layers.Dense(784, activation=None),
    ])
    x = ed.Bernoulli(gen_net(z))
    return x

def variational_model(x):
    inference_net = tf.keras.Sequential([
        tf.keras.layers.Dense(100, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=None),
    ])
    z = ed.Normal(loc=inference_net(x), scale=tf.ones([28*28]), name='z_posterior')
    return z

def replace_z(z):
    def interceptor(rv_constructor, *rv_args, **rv_kwargs):
        name = rv_kwargs.pop('name')
        if name == 'z':
            rv_kwargs['value'] = z
        return rv_constructor(*rv_args, **rv_kwargs)
    return interceptor

log_joint = ed.make_log_joint_fn(generative_model)
def loss_fn(x):
    z = variational_model(x)
    energy = log_joint(z=z, x=x)
    entropy = tf.reduce_sum(z.entropy())
    return -energy-entropy
opt = tf.contrib.opt.AdamWOptimizer(0, learning_rate=1e-3)

In [78]:
# load MNIST.
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
proc = lambda x: np.rint((x / 255.).reshape(-1, 28*28)).astype(np.int32)
x_train, x_test = proc(x_train), proc(x_test)
assert x_train.max() <= 1 and x_train.min() >= 0

In [79]:
batch = x_train[:10]
print(batch.dtype)
x_ph = tf.placeholder(tf.float32, [None, 28*28])
loss = loss_fn(x_ph)
with tf.Session() as sess:
    print(sess.run(loss, {x_ph: batch}))

int32


/home/deploy/.local/share/virtualenvs/edward2-notebooks-Fuhq9eE6/lib/python3.5/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  return _inspect.getargspec(target)
/home/deploy/.local/share/virtualenvs/edward2-notebooks-Fuhq9eE6/lib/python3.5/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  return _inspect.getargspec(target)
/home/deploy/.local/share/virtualenvs/edward2-notebooks-Fuhq9eE6/lib/python3.5/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  return _inspect.getargspec(target)
/home/deploy/.local/share/virtualenvs/edward2-notebooks-Fuhq9eE6/lib/python3.5/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
 

ValueError: Incompatible shapes for broadcasting: (?, 10) and (784,)